### USCRN Data: High-Octane Scraping

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd 
import yaml 
import re
import itertools

with open ("sources.yaml", "r") as yaml_file:
  sources = yaml.load(yaml_file, Loader=yaml.FullLoader)

##### 1.) Scrape Column Headers and Descriptions 

In [3]:
header_url = sources['USCRN']['headers']
header_response = requests.get(header_url)
header_soup = BeautifulSoup(header_response.content, "html.parser")

columns = str(header_soup).split("\n")[1].strip(" ").split(" ")
columns = list(map(lambda x: str.lower(x), columns)) # columns = [str.lower(c) for c in columns] -- faster?
columns.insert(0,'station_location')

descrip_text = str(header_soup).split("\n")[2] # raw text block containing column descriptions
descrip_text

"The station WBAN number. The UTC date of the observation. The UTC time of the observation. Time is the end of the observed hour, so the 0000 hour is actually the last hour of the previous day's observation (starting just after 11:00 PM through midnight). The Local Standard Time (LST) date of the observation. The Local Standard Time (LST) time of the observation. Time is the end of the observed hour (see UTC_TIME description). The version number of the station datalogger program that was in effect at the time of the observation. Note: This field should be treated as text (i.e. string). Station longitude, using WGS-84. Station latitude, using WGS-84. Average air temperature, in degrees C, during the last 5 minutes of the hour. See Note F. Average air temperature, in degrees C, for the entire hour. See Note F. Maximum air temperature, in degrees C, during the hour. See Note F. Minimum air temperature, in degrees C, during the hour. See Note F. Total amount of precipitation, in mm, record

The descriptions of the columns are quite the mess, as there is no standard separator used. We will have to work our way through it step by step: 

In [4]:
first_split = [re.sub(r'(\([^)]*)$', r"\1)", s) for s in descrip_text.split("). ")] # add ')' back after splitting text on ').' 
no_notes = [re.sub(r' See Note [A-Z]\.',"",s) for s in first_split] # drop any references to notes

The third entry in `no_notes` is ready. The last set of descriptions in `no_notes` can be split on `". "`, but the first two sets need special attention. We will pop the last set out and split it, then pop the third set out, and then address the first two sets. At that point we will recombine everything into one list while preserving the original order. 

In [5]:
last_set = no_notes.pop().strip().split(". ")
third_set = no_notes.pop() # Note: just a string

In [6]:
def flatten(ls:list): 
  return list(itertools.chain.from_iterable(ls)) 

no_notes = [re.sub(". Time is", " at", s) for s in no_notes]
first_second = flatten([s.split(". ") for s in no_notes])

# Finally:
descriptions = flatten([["Location name for USCRN station"], first_second, [third_set], last_set]) # Description added for "station_location" 

In [15]:
header_info = {
  'col_name': columns,
  'description': descriptions, 
  'units': ["X...(Various Lengths)","XXXXX", "YYYYMMDD", "HHmm", "YYYYMMDD", "HHmm", "XXXXXX", "Decimal_degrees", "Decimal_degrees", "Celsius", "Celsius", "Celsius", "Celsius", "mm", "W/m^2", "X", "W/m^2", "X", "W/m^2", "X", "X", "Celsius", "X", "Celsius", "X", "Celsius", "X", "%", "X", "m^3/m^3", "m^3/m^3", "m^3/m^3", "m^3/m^3", "m^3/m^3", "Celsius", "Celsius", "Celsius", "Celsius", "Celsius"]
}

header_df = pd.DataFrame(header_info)
header_df.to_csv("data/column_descriptions.csv", index=True)

##### 2.) Scrape Core Data Files (>2 million rows)

In [3]:
base_url = sources["USCRN"]["index"]
base_response = requests.get(base_url)
base_soup = BeautifulSoup(base_response.content, "html.parser")

In [15]:
links = base_soup.find_all("a") # 'links' in this notebook will refer to <a> elements, not urls
years = [str(x).zfill(1) for x in range(2000,2024)]
year_links = [link for link in links if link['href'].rstrip('/') in years]

file_urls = []
for year_link in year_links: 
  year_url = base_url + year_link.get("href")
  response = requests.get(year_url) 
  soup = BeautifulSoup(response.content, 'html.parser')
  file_links = soup.find_all('a', href=re.compile(r'AK.*\.txt'))
  if file_links:
    new_file_urls = [year_url + link.getText() for link in file_links]
    file_urls.extend(new_file_urls)

In [5]:
with open("file_urls.txt", "r") as file: 
  file_urls = [s.strip() for s in file.readlines()]

In [6]:
rows = []
regex = r"([St.]*[A-Z][a-z]+_*[A-Za-z]*).*.txt" 
for url in file_urls:
  # Get location from url -- will add to BS results in next step
  file_name = re.search(regex, url).group(0)
  station_location = re.sub("(_formerly_Barrow.*|_[0-9].*)", "", file_name)
  # Get results 
  response = requests.get(url)
  soup = BeautifulSoup(response.content,'html.parser')
  soup_lines = [station_location + " " + line for line in str(soup).strip().split("\n")]
  #   soup_lines = [station_location + " " + line for line in str(soup).split("\n")] <-- Old Code 
  new_rows = [re.split('\s+', row) for row in soup_lines]
  rows.extend(new_rows)

In [7]:
df = pd.DataFrame(rows, columns=columns)

In [8]:
df.sample(10)

,station_location,wbanno,utc_date,utc_time,lst_date,lst_time,crx_vn,longitude,latitude,t_calc,...,soil_moisture_5,soil_moisture_10,soil_moisture_20,soil_moisture_50,soil_moisture_100,soil_temp_5,soil_temp_10,soil_temp_20,soil_temp_50,soil_temp_100
1020139,Gustavus,25380,20170705,0200,20170704,1700,2.514,-135.69,58.43,12.1,...,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
779493,Selawik,96407,20151112,0900,20151112,0000,2.515,-159.00,66.56,-12.6,...,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
1678063,Selawik,96407,20200808,1200,20200808,0300,2.515,-159.00,66.56,11.5,...,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
803258,Tok,96404,20150730,1400,20150730,0500,2.514,-141.21,62.74,6.7,...,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
505254,Sand_Point,25630,20131020,1800,20131020,0900,2.424,-160.47,55.35,3.0,...,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
771982,Sand_Point,25630,20150523,1300,20150523,0400,2.424,-160.47,55.35,5.0,...,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
2153138,Ruby,96406,20230112,0200,20230111,1700,2.515,-154.13,64.50,-11.7,...,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
561521,Glennallen,56401,20140917,2300,20140917,1400,2.515,-145.50,63.03,9.8,...,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
1532475,Aleknagik,23583,20200111,0800,20200110,2300,2.514,-158.61,59.28,-13.6,...,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
891697,Metlakatla,25381,20160825,1300,20160825,0400,2.424,-131.59,55.05,16.1,...,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0


In [12]:
# Check for NaNs
df[df.isna().any(axis=1)]

,0,1,2,3,4,5,6,7,8,9,...,29,30,31,32,33,34,35,36,37,38


Let's drop the soil temperature and moisture columns. These fields are erroneous (-99.0 or -9999.0) for the vast majority of records.

In [9]:
soil_columns = df.filter(regex="soil")
soil_columns[soil_columns.isin([-99.0, -9999.0]).any(axis=1)]
soil_columns

,soil_moisture_5,soil_moisture_10,soil_moisture_20,soil_moisture_50,soil_moisture_100,soil_temp_5,soil_temp_10,soil_temp_20,soil_temp_50,soil_temp_100
0,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
1,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
2,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
3,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
4,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
...,...,...,...,...,...,...,...,...,...,...
2162938,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
2162939,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
2162940,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
2162941,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0


In [10]:
df = df.filter(regex="^((?!soil).)*$")

The `solar_rad` and `sur_temp` sets of columns also have some erroneous measurements, but to a much lesser extent. We will keep these.

In [ ]:
# Saving dataframe
df.to_csv("data/uscrn.csv", index=False)

Let's also make a table for our various station locations. This will be useful when searching for the four-day forecasts in the NWS notebook. 

In [1]:
# import pandas as pd 
# df = pd.read_csv("data/uscrn.csv")
locations = df[['station_location', 'wbanno', 'longitude', 'latitude']].drop_duplicates()
locations.to_csv("data/locations.csv", index=False)

##### 3.) Upload to BigQuery 

In [79]:
%%bash
bq mk -d --location=us-east4 team-week3:alaska

Dataset 'team-week3:alaska' successfully created.


Core Data: 

In [1]:
from google.cloud import bigquery
from google.oauth2 import service_account

# Manually set schema to make columns required (NaN detection on upload) and avoid parsing certain columns as floats (e.g. date columns)
schema = [
  bigquery.SchemaField("station_location", "STRING", mode="REQUIRED"), 
  bigquery.SchemaField("wbanno", "STRING", mode="REQUIRED"),  
  bigquery.SchemaField("utc_date", "STRING", mode="REQUIRED"), # Not sure if BigQuery would be able to 
  bigquery.SchemaField("utc_time", "STRING", mode="REQUIRED"), # read these date columns as dates 
  bigquery.SchemaField("lst_date", "STRING", mode="REQUIRED"), # given their format.
  bigquery.SchemaField("lst_time", "STRING", mode="REQUIRED"), 
  bigquery.SchemaField("crx_vn", "STRING", mode="REQUIRED"), 
  bigquery.SchemaField("longitude", "FLOAT", mode="REQUIRED"), 
  bigquery.SchemaField("latitude", "FLOAT", mode="REQUIRED"), 
  bigquery.SchemaField("t_calc", "FLOAT", mode="REQUIRED"), 
  bigquery.SchemaField("t_hr_avg", "FLOAT", mode="REQUIRED"), 
  bigquery.SchemaField("t_max", "FLOAT", mode="REQUIRED"), 
  bigquery.SchemaField("t_min", "FLOAT", mode="REQUIRED"), 
  bigquery.SchemaField("p_calc", "FLOAT", mode="REQUIRED"), 
  bigquery.SchemaField("solarad", "FLOAT", mode="REQUIRED"), 
  bigquery.SchemaField("solarad_flag", "STRING", mode="REQUIRED"), 
  bigquery.SchemaField("solarad_max", "FLOAT", mode="REQUIRED"), 
  bigquery.SchemaField("solarad_max_flag", "STRING", mode="REQUIRED"), 
  bigquery.SchemaField("solarad_min", "FLOAT", mode="REQUIRED"), 
  bigquery.SchemaField("solarad_min_flag", "STRING", mode="REQUIRED"), 
  bigquery.SchemaField("sur_temp_type", "STRING", mode="REQUIRED"), 
  bigquery.SchemaField("sur_temp", "FLOAT", mode="REQUIRED"), 
  bigquery.SchemaField("sur_temp_flag", "STRING", mode="REQUIRED"), 
  bigquery.SchemaField("sur_temp_max", "FLOAT", mode="REQUIRED"), 
  bigquery.SchemaField("sur_temp_max_flag", "STRING", mode="REQUIRED"), 
  bigquery.SchemaField("sur_temp_min", "FLOAT", mode="REQUIRED"), 
  bigquery.SchemaField("sur_temp_min_flag", "STRING", mode="REQUIRED"), 
  bigquery.SchemaField("rh_hr_avg", "FLOAT", mode="REQUIRED"), 
  bigquery.SchemaField("rh_hr_avg_flag", "STRING", mode="REQUIRED"), 
]


In [3]:
key_path = ""
credentials = service_account.Credentials.from_service_account_file(
   key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

client = bigquery.Client(credentials=credentials, project=credentials.project_id)

table_id = f"{credentials.project_id}.alaska.uscrn"

job_config = bigquery.LoadJobConfig(
   source_format="CSV",
   schema=schema,
   create_disposition="CREATE_IF_NEEDED",
   write_disposition="WRITE_TRUNCATE", 
   description="Historical weather data taken at USCRN stations in Alaska"   
)

with open("data/uscrn.csv", "rb") as fp:
   job = client.load_table_from_file(fp, table_id, job_config=job_config)
job.result() # wait for the load job to complete


SyntaxError: invalid syntax (1736132934.py, line 21)

(For now, not uploading `column_descriptions.csv`)